# Domaine des Croix

Le client, le Domaine des Croix, cherche à se lancer sur le marché américain . Il souhaite donc **définir le prix** de ses bouteilles de vin **pour être compétitif sur le marché américain**. Il a récupéré un jeu de données de 130k bouteilles de vin, avec les cépages, les pays et région de production, les millésimes (c'est-à-dire les années de production), ainsi que des notes ("points") et descriptifs d'oenologues (les spécialistes du vin), et le prix moyen en dollars de toutes ces bouteilles sur le marché américain.


## Jeux de données
- Dataset des 130k vins : https://github.com/WildCodeSchool/wilddata/raw/main/wine.zip
- Dataset de la bouteille de vin que le client aimerait proposer sur le marché américain : https://raw.githubusercontent.com/WildCodeSchool/wilddata/main/domaine_des_croix.csv


## Exploration et nettoyage des données

In [24]:
import pandas as pd

link = "https://github.com/WildCodeSchool/wilddata/raw/main/wine.zip"
df_wine = pd.read_csv(link)

df_wine.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2019 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,20.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2017 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,18.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2019 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,18.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2019 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2016, this...",Vintner's Reserve Wild Child Block,87,44.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2018 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [17]:
# J'importe le dataset du client

data_ddc = "https://raw.githubusercontent.com/WildCodeSchool/wilddata/main/domaine_des_croix.csv"

df_ddc = pd.read_csv(data_ddc)

df_ddc

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,France,Firm and solid with great swathes of black fru...,NaN,94,NaN,Burgundy,Corton Grèves,NaN,Roger Voss,@vossroger,Domaine des Croix 2016 Corton Grèves,Pinot Noir,Domaine des Croix


In [25]:
# Je commence par une analyse rapide de mon dataset

df_wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129957 entries, 0 to 129956
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   country                129894 non-null  object 
 1   description            129957 non-null  object 
 2   designation            92497 non-null   object 
 3   points                 129957 non-null  int64  
 4   price                  120964 non-null  float64
 5   province               129894 non-null  object 
 6   region_1               108710 non-null  object 
 7   region_2               50511 non-null   object 
 8   taster_name            103713 non-null  object 
 9   taster_twitter_handle  98744 non-null   object 
 10  title                  129957 non-null  object 
 11  variety                129956 non-null  object 
 12  winery                 129957 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 12.9+ MB


### Premièr coup d'oeil

- Presque 130000 références de vin
- Beaucoup de valeurs manquantes
- Certaines colonnes ne nous seront pas utiles 
- Pas d'erreur visible sur les types de données

In [19]:
# Affichage des statistiques descriptives
df_wine.describe()

,points,price
count,129957.000000,120964.000000
mean,88.446655,36.390827
std,3.039481,32.802430
min,80.000000,2.000000
25%,86.000000,21.000000
50%,88.000000,29.000000
75%,91.000000,42.000000
max,100.000000,1902.000000


Concernant les points (notes) :
- Très bonne moyenne de 88 points
- Ecart type faible
- Aucun vin noté en dessous de 80 points

Concernant les prix : 
- Grande variabilité des prix si l'on juge la moyenne et l'écart-type.
Le (ou les) outliers y sont pour quelque chose
- 75% des vins sont en dessous de 42$, c'est autant de
concurence en moins selon notre stratégie de positionnement
- Présence de vins d'exceptions (outliers) à 1902$ 
la bouteille qui ne viendra pas gêner notre analyse

In [27]:
# Je regarde mes valeurs nulles

df_wine.isna().sum()

country                     63
description                  0
designation              37460
points                       0
price                     8993
province                    63
region_1                 21247
region_2                 79446
taster_name              26244
taster_twitter_handle    31213
title                        0
variety                      1
winery                       0
dtype: int64

Les cellules à faibles valeurs : 
- region_2, avec plus de 60% de valeurs nulles
- taster_name, je ne vais pas m'en servir pour mon analyse
- taster_twitter-handle, encore moins

Je garde mes autres colonnes, certaines me seront peut-être utiles
pour la mise en forme de mon Dashboard.



# Nettoyage du dataset

In [28]:
# Je commence par mettre de côté mes colonnes à faible valeur 

df_wine = df_wine.drop(['taster_twitter_handle', 'taster_name', 'region_2'], axis=1)

In [ ]:
# Petite vérification
df_wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129957 entries, 0 to 129956
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   country      129894 non-null  object 
 1   description  129957 non-null  object 
 2   designation  92497 non-null   object 
 3   points       129957 non-null  int64  
 4   price        120964 non-null  float64
 5   province     129894 non-null  object 
 6   region_1     108710 non-null  object 
 7   title        129957 non-null  object 
 8   variety      129956 non-null  object 
 9   winery       129957 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 9.9+ MB


In [30]:
# Grâce à une Regex, j'extrait mes millesimes de la colonne title
# que je stocke dans une nouvelle colonne ['millesime']

import re

df_wine['millesime'] = df_wine['title'].str.extract(r"(\d{4})").astype(str)

In [31]:
# Je me sépare de toutes les bouteilles étiquettées 'champagne'
# Je fais un masque booleen pour conserver l'inverse de ma condition

df_wine = df_wine[~df_wine['title'].str.contains("champagne", case=False, na=False)]

In [32]:
# Je supprime les lignes sans pays et region
df_wine = df_wine[df_wine['country'].notna()]

In [33]:
# Je supprime les lignes sans prix
df_wine = df_wine.dropna(subset=['price'])

In [34]:
# Petit point vérification
df_wine.info()

<class 'pandas.core.frame.DataFrame'>
Index: 119516 entries, 1 to 129956
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   country      119516 non-null  object 
 1   description  119516 non-null  object 
 2   designation  84753 non-null   object 
 3   points       119516 non-null  int64  
 4   price        119516 non-null  float64
 5   province     119516 non-null  object 
 6   region_1     100000 non-null  object 
 7   title        119516 non-null  object 
 8   variety      119515 non-null  object 
 9   winery       119516 non-null  object 
 10  millesime    119516 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 10.9+ MB


In [35]:
# Je rempli mes millesimes 'nan' par 'NC' (Non communiqué)
df_wine['millesime'] = df_wine['millesime'].fillna("NC").astype(str)

In [36]:
# Je rempli toutes mes autres valeurs textuels vide par NC également
df_wine = df_wine.fillna('NC')


In [37]:
# Je regarde si j'ai des doublons
df_wine.duplicated().sum()

9277

In [38]:
# 9277 doublons, je supprime
df_wine = df_wine.drop_duplicates()

In [39]:
# Point vérification
df_wine.info()

<class 'pandas.core.frame.DataFrame'>
Index: 110239 entries, 1 to 129956
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   country      110239 non-null  object 
 1   description  110239 non-null  object 
 2   designation  110239 non-null  object 
 3   points       110239 non-null  int64  
 4   price        110239 non-null  float64
 5   province     110239 non-null  object 
 6   region_1     110239 non-null  object 
 7   title        110239 non-null  object 
 8   variety      110239 non-null  object 
 9   winery       110239 non-null  object 
 10  millesime    110239 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 10.1+ MB


In [40]:
# Je reset mes index pour être bien propre
df_wine.reset_index(drop=True, inplace=True)

In [41]:
# J'affiche mon dataset optimisé
df_wine.head()

,country,description,designation,points,price,province,region_1,title,variety,winery,millesime
0,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,20.0,Douro,NC,Quinta dos Avidagos 2017 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2017
1,US,"Tart and snappy, the flavors of lime flesh and...",NC,87,18.0,Oregon,Willamette Valley,Rainstorm 2019 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2019
2,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,18.0,Michigan,Lake Michigan Shore,St. Julian 2019 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2019
3,US,"Much like the regular bottling from 2016, this...",Vintner's Reserve Wild Child Block,87,44.0,Oregon,Willamette Valley,Sweet Cheeks 2018 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2018
4,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,20.0,Northern Spain,Navarra,Tandem 2017 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,2017


In [ ]:
# Mon dataset est désormais nettoyé et optimisé, j'exporte...
df_wine.to_csv('data_wine.csv', index=False)

## Conclusion

- Je passe d'un dataset de presque 130000 références à 110000.
- Le nettoyage à permis de conserver **85%** de mes données initiales. 